Datastore basic tutorial
- print works
- Warning message: /opt/homebrew/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)

In [ ]:
# # Imports the Google Cloud client library
# from google.cloud import datastore

# # debug OSError: project was not passed
# import os
# os.environ["GCLOUD_PROJECT"] = "datastore-tutorial-445100"

# # Instantiates a client
# datastore_client = datastore.Client()

# # The kind for the new entity
# kind = "Task"
# # The name/ID for the new entity
# name = "sampletask1"
# # The Cloud Datastore key for the new entity
# task_key = datastore_client.key(kind, name)

# # Prepares the new entity
# task = datastore.Entity(key=task_key)
# task["description"] = "Buy milk"

# # Saves the entity
# datastore_client.put(task)

# print(f"Saved {task.key.name}: {task['description']}")

/opt/homebrew/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Saved sampletask1: Buy milk


to create a new entity, we need
- kind: table / collection name
- a unique name: could be created by concatenating elements (this will be 1st col)
- then with kind and name, we create a key
- then we create Entity with the key

Note: Use Python 3.9.19 kernel

In [28]:
# from flask import Flask, request, jsonify
from flask import Flask
from google.cloud import datastore # datastore mode
from google.appengine.api import wrap_wsgi_app
from google.appengine.api import memcache # memcache for efficiency
from datetime import datetime

In [33]:
app = Flask(__name__)
app.wsgi_app = wrap_wsgi_app(app.wsgi_app, use_legacy_context_mode=True, use_deferred=False)

In [34]:
# Instantiates a client
datastore_client = datastore.Client()

/opt/homebrew/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
# log daily habit into the database

@app.route('/log_habit', methods=['POST'])
def log_habit():
    # with front end, comment out for now
    # user_id = request.form.get("user_id")
    # habit_id = request.form.get("habit_id")
    # today = datetime.now()
    
    habit_kind = "HabitLog"
    
    
    user_id = "Alice"
    habit_id = "Exercise"
    today = datetime.now()

    # keys for memcache
    streak_cache_key = f"streak_{user_id}_{habit_id}"
    month_cache_key = f"calendar_{user_id}_{habit_id}_{today.month}_{today.year}"
    print(streak_cache_key)
    print(month_cache_key)

    # update database
    # key = datastore_client.key(kind, name)
    log_key = datastore_client.key(habit_kind, f"{user_id}_{habit_id}_{today}")
    log = datastore.Entity(key=log_key)
    log.update({"user_id": user_id, "habit_id": habit_id, "last_log_date": today})
    # saves the entity
    datastore_client.put(log)

    # Update Streak in Memcache
    # streak = calculate_streak(user_id, habit_id)
    streak = 1
    memcache.set(streak_cache_key, streak, time=3600) # save for 1 hr

    # Update Logged Days in Memcache
    logged_days = memcache.get(month_cache_key) or set()
    logged_days.add(today.day)
    memcache.set(month_cache_key, logged_days, time=3600)

    return jsonify({"message": "Habit logged successfully", "streak": streak})

In [36]:
log_habit()

streak_Alice_Exercise
calendar_Alice_Exercise_12_2024


RPCFailedError: Attempted RPC call without active security ticket